## Importing Libraries

In [1]:
!pip install tensorflow 
!pip install gym
!pip install keras
!pip install keras-rl2  # For reinforcemenet learning

In [31]:
import gym
import random

## Environment

Import CartPole environment from openAI gym.

In [32]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
print(f'Total Numbers of states are {states}')
actions = env.action_space.n
print(f'Total Number of actions are {actions}')

Total Numbers of states are 4
Total Number of actions are 2


Let's test the environment by applying random action [0,1] for [Left,Right]. 

In [39]:
episodes = 30
for episode in range(episodes):
    state = env.reset()  # Reseting environment to its initial position
    done = False
    score = 0 
    
    while not done:
        env.render()    # To Display
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print(f'Episode:{episode+1} Score:{score}')

Episode:1 Score:25.0
Episode:2 Score:12.0
Episode:3 Score:31.0
Episode:4 Score:34.0
Episode:5 Score:34.0
Episode:6 Score:14.0
Episode:7 Score:28.0
Episode:8 Score:54.0
Episode:9 Score:39.0
Episode:10 Score:56.0
Episode:11 Score:32.0
Episode:12 Score:25.0
Episode:13 Score:29.0
Episode:14 Score:25.0
Episode:15 Score:24.0
Episode:16 Score:31.0
Episode:17 Score:24.0
Episode:18 Score:17.0
Episode:19 Score:49.0
Episode:20 Score:23.0
Episode:21 Score:12.0
Episode:22 Score:30.0
Episode:23 Score:20.0
Episode:24 Score:12.0
Episode:25 Score:81.0
Episode:26 Score:30.0
Episode:27 Score:20.0
Episode:28 Score:20.0
Episode:29 Score:32.0
Episode:30 Score:14.0


## Building a Model

In [34]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [35]:
model = Sequential(   
    [
    Flatten(input_shape=(1,states)),
    Dense(24, activation='relu'),
    Dense(24, activation='relu'),
    Dense(actions, activation='linear')
    ]
)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


## Building an RL model

In [40]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy  # for policy
from rl.memory import SequentialMemory   # for storing values

In [37]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

Let's compile and train the DQN model.

In [38]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 74s 7ms/step - reward: 1.0000
102 episodes - episode_reward: 96.647 [10.000, 379.000] - loss: 1.808 - mae: 18.511 - mean_q: 37.558

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 75s 8ms/step - reward: 1.0000
45 episodes - episode_reward: 223.467 [157.000, 371.000] - loss: 2.790 - mae: 38.257 - mean_q: 77.278

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 76s 8ms/step - reward: 1.0000
40 episodes - episode_reward: 251.050 [157.000, 404.000] - loss: 1.931 - mae: 42.701 - mean_q: 86.079

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 78s 8ms/step - reward: 1.0000
39 episodes - episode_reward: 251.282 [187.000, 437.000] - loss: 1.543 - mae: 43.777 - mean_q: 88.074

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 78s 8ms/step - reward: 1.0000
done

Now the traning is done. Let's see how well the model is performing.

In [41]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 240.000, steps: 240
Episode 2: reward: 238.000, steps: 238
Episode 3: reward: 288.000, steps: 288
Episode 4: reward: 237.000, steps: 237
Episode 5: reward: 216.000, steps: 216
Episode 6: reward: 271.000, steps: 271
Episode 7: reward: 306.000, steps: 306
Episode 8: reward: 215.000, steps: 215
Episode 9: reward: 293.000, steps: 293
Episode 10: reward: 233.000, steps: 233
253.7


Awesome! The performance is outstanding, in 200 step model is able to take good decision is every step and collected 200 rewards.